In [ ]:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install requests beautifulsoup4 pandas

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium


Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Err:1 https://dl.google.com/linux/chrome/deb stable InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 32EE5355A6BC6E42
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: GPG e

In [ ]:
# Hapus versi yang sekarang dulu
!apt-get remove -y google-chrome-stable

# Install ulang versi 114
!wget https://mirror.cs.uchicago.edu/google-chrome/pool/main/g/google-chrome-stable/google-chrome-stable_114.0.5735.90-1_amd64.deb
!apt-get install -y ./google-chrome-stable_114.0.5735.90-1_amd64.deb

# Cegah supaya gak auto-upgrade
!apt-mark hold google-chrome-stable


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libu2f-udev libvulkan1 mesa-vulkan-drivers
Use 'apt autoremove' to remove them.
The following packages will be REMOVED:
  google-chrome-stable
The following held packages will be changed:
  google-chrome-stable
0 upgraded, 0 newly installed, 1 to remove and 36 not upgraded.
E: Held packages were changed and -y was used without --allow-change-held-packages.
--2025-06-22 10:45:02--  https://mirror.cs.uchicago.edu/google-chrome/pool/main/g/google-chrome-stable/google-chrome-stable_114.0.5735.90-1_amd64.deb
Resolving mirror.cs.uchicago.edu (mirror.cs.uchicago.edu)... 128.135.164.21
Connecting to mirror.cs.uchicago.edu (mirror.cs.uchicago.edu)|128.135.164.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93841272 (89M) [application/octet-stream]
Saving to: ‘google-chrome-stable_114.0.5735

In [ ]:
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip -o chromedriver_linux64.zip
!mv -f chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver

--2025-06-22 10:46:17--  https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 74.125.20.207, 108.177.98.207, 74.125.135.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7407250 (7.1M) [application/zip]
Saving to: ‘chromedriver_linux64.zip.2’

chromedriver_linux6 100%[===================>]   7.06M  --.-KB/s    in 0.08s   

2025-06-22 10:46:17 (91.6 MB/s) - ‘chromedriver_linux64.zip.2’ saved [7407250/7407250]

Archive:  chromedriver_linux64.zip
  inflating: chromedriver            
  inflating: LICENSE.chromedriver    


In [ ]:
!google-chrome --version
!chromedriver --version


Google Chrome 114.0.5735.90 
ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from datetime import datetime, timedelta
import time
import os

# Setup opsi Chrome
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service("/usr/bin/chromedriver"), options=chrome_options)

# Bulan dalam Bahasa Indonesia
bulan_id = {
    1: "Januari", 2: "Febuari", 3: "Maret", 4: "April",
    5: "Mei", 6: "Juni", 7: "Juli", 8: "Agustus",
    9: "September", 10: "Oktober", 11: "November", 12: "Desember"
}

# Range waktu
start_date = datetime(2020, 1, 1)
end_date = datetime(2020, 4, 1)

# Lokasi penyimpanan
save_dir = "/content/drive/MyDrive/2025_AI/scrape-gold"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "harga_emas_xpath.csv")
cache_path = os.path.join(save_dir, "cache_harga_emas.csv")

# Load cache jika ada
data = []
if os.path.exists(cache_path):
    df = pd.read_csv(cache_path)
    data = df.to_dict(orient="records")
    if data:
        last_date = datetime.strptime(data[-1]["date"], "%Y-%m-%d")
        start_date = last_date + timedelta(days=1)
        print(f"📂 Melanjutkan dari cache terakhir: {last_date.date()}")
else:
    df = pd.DataFrame(data)

# XPath mapping
xpaths = {
    "harga_oz": "/html/body/div[1]/div[3]/div[1]/table[1]/tbody/tr[3]/td[2]",
    "kurs_oz":  "/html/body/div[1]/div[3]/div[1]/table[1]/tbody/tr[3]/td[3]",
    "idr_oz":   "/html/body/div[1]/div[3]/div[1]/table[1]/tbody/tr[3]/td[4]",
    "harga_gr": "/html/body/div[1]/div[3]/div[1]/table[1]/tbody/tr[4]/td[2]",
    "kurs_gr":  "/html/body/div[1]/div[3]/div[1]/table[1]/tbody/tr[4]/td[3]",
    "idr_gr":   "/html/body/div[1]/div[3]/div[1]/table[1]/tbody/tr[4]/td[4]",
}

# Mulai scraping
while start_date < end_date:
    bulan = bulan_id[start_date.month]
    url = f"https://harga-emas.org/history-harga/{start_date.year}/{bulan}/{start_date.day}/"
    print(f"🔗 {url}")

    try:
        driver.get(url)

        # Polling ringan (tanpa WebDriverWait)
        success = False
        for _ in range(5):
            try:
                if driver.find_elements(By.CLASS_NAME, "in_table"):
                    success = True
                    break
            except Exception:
                pass
            time.sleep(1)

        if not success:
            raise Exception("⏱️ Timeout menunggu elemen .in_table")

        row = {"date": start_date.strftime("%Y-%m-%d")}

        for key, xpath in xpaths.items():
            try:
                el = driver.find_elements(By.XPATH, xpath)
                row[key] = el[0].text.strip() if el else "N/A"
            except NoSuchElementException:
                row[key] = "N/A"

        data.append(row)
        df = pd.DataFrame(data)
        df.to_csv(cache_path, index=False)  # Simpan cache progres
        print(f"✅ {start_date.strftime('%Y-%m-%d')} | Sukses")

    except Exception as e:
        print(f"❌ Gagal scrape {start_date.strftime('%Y-%m-%d')}: {e}")

    start_date += timedelta(days=1)

driver.quit()

# Simpan final
df.to_csv(save_path, index=False)
print(f"\n📁 Data akhir tersimpan: {save_path}")
print(f"📊 Total baris data: {len(df)}")


🔗 https://harga-emas.org/history-harga/2020/Januari/1/
✅ 2020-01-01 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/2/
✅ 2020-01-02 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/3/
✅ 2020-01-03 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/4/
✅ 2020-01-04 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/5/
✅ 2020-01-05 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/6/
✅ 2020-01-06 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/7/
✅ 2020-01-07 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/8/
✅ 2020-01-08 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/9/
✅ 2020-01-09 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/10/
✅ 2020-01-10 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/11/
✅ 2020-01-11 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/12/
✅ 2020-01-12 | Sukses
🔗 https://harga-emas.org/history-harga/2020/Januari/13/
✅ 2020-01-13 | Su

In [ ]:
#untuk hapus cache csv ( jika terjadi kegagalan )
import os

cache_path = "/content/drive/MyDrive/2025_AI/scrape-gold/cache_harga_emas.csv"

if os.path.exists(cache_path):
    os.remove(cache_path)
    print("🗑️ Cache berhasil dihapus.")
else:
    print("ℹ️ Tidak ada file cache untuk dihapus.")


🗑️ Cache berhasil dihapus.


VERSI PAKE BEAUTIFOUL SOUP SAMA REQUEST , TANPA SELENIUM :

In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv

# --- Konfigurasi bulan Indonesia ---
bulan_id = {
    1: "Januari", 2: "Febuari", 3: "Maret", 4: "April",
    5: "Mei", 6: "Juni", 7: "Juli", 8: "Agustus",
    9: "September", 10: "Oktober", 11: "November", 12: "Desember"
}

# Rentang waktu scraping
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 3, 13)

headers = {"User-Agent": "Mozilla/5.0"}

# --- Fungsi scraping per hari ---
def scrape_per_hari(tahun, bulan, tanggal):
    nama_bulan = bulan_id[bulan]
    url = f"https://harga-emas.org/history-harga/{tahun}/{nama_bulan}/{tanggal:02d}/"
    try:
        r = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(r.content, "html.parser")
        rows = soup.find_all("tr")

        if len(rows) < 12:
            return None

        # Ambil baris ounce dan gram berdasarkan asumsi posisi tetap
        oz_cells = rows[10].find_all("td")
        gr_cells = rows[11].find_all("td")

        harga_oz = oz_cells[1].text.strip()
        kurs_oz = oz_cells[2].text.strip()
        idr_oz = oz_cells[3].text.strip()

        harga_gr = gr_cells[1].text.strip()
        kurs_gr = gr_cells[2].text.strip()
        idr_gr = gr_cells[3].text.strip()

        # Ambil tanggal update
        update_raw = ""
        for row in rows:
            if "Update harga emas" in row.text:
                update_raw = row.text.strip()
                break

        update_tanggal = update_raw.split("Update harga emas :")[-1].split("Update kurs")[0].strip()
        tanggal_label = f"{tanggal:02d} {nama_bulan} {tahun}"

        return {
            "tanggal": tanggal_label,
            "harga_oz": harga_oz,
            "kurs_oz": kurs_oz,
            "idr_oz": idr_oz,
            "harga_gr": harga_gr,
            "kurs_gr": kurs_gr,
            "idr_gr": idr_gr,
            "update_asli": update_tanggal
        }
    except Exception as e:
        print(f"Gagal scraping {tanggal:02d}-{nama_bulan}-{tahun}: {e}")
        return None

# --- Loop utama ---
results = []
current = start_date
while current <= end_date:
    nama_bulan = bulan_id[current.month]
    print(f"Memproses {current.day:02d} {nama_bulan} {current.year}...")
    data = scrape_per_hari(current.year, current.month, current.day)
    if data:
        results.append(data)
    current += timedelta(days=1)

# --- Simpan hasil ke CSV ---
with open("harga_emas_febuari_2020-2025.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=results[0].keys())
    writer.writeheader()
    writer.writerows(results)

print("✅ Selesai menyimpan data ke harga_emas_febuari_2020-2025.csv")


🆕 Cache tidak ditemukan, memulai scraping dari awal.
🔗 Mengambil data dari: https://harga-emas.org/history-harga/2020/Januari/1/
✅ 2020-01-01 | Data berhasil diambil dan cache diperbarui.
🔗 Mengambil data dari: https://harga-emas.org/history-harga/2020/Januari/2/
✅ 2020-01-02 | Data berhasil diambil dan cache diperbarui.
🔗 Mengambil data dari: https://harga-emas.org/history-harga/2020/Januari/3/
✅ 2020-01-03 | Data berhasil diambil dan cache diperbarui.

📁 Data akhir tersimpan di: /content/drive/MyDrive/2025_AI/scrape-gold/harga_emas_requests.csv
📊 Total baris data: 3


In [ ]:
rm /content/drive/MyDrive/2025_AI/scrape-gold/cache_harga_emas_requests.csv